# A04-航空旅客付费选座意愿

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import re

In [2]:
data = pd.read_csv('data.csv',encoding='utf-8')#读取文件
passport=data['pax_passport']

/Users/hanpeitao/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (48,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.seg_cabin.isnull().sum()#唯一一个nan

1

In [4]:
data.fillna(0,inplace=True)

# 根据3σ原则修改异常值

In [6]:
int_col = [col for col in data if data[col].dtypes!=object]
int_col.pop(int_col.index('ffp_nbr'))
int_col.pop(int_col.index('emd_lable'))
int_col.pop(int_col.index('emd_lable2'))
int_col.pop(int_col.index('often_city'))

'often_city'

In [7]:
for i in range(122-97):
    int_col.pop(i+97)

In [8]:
n=3
int_col=['pax_tax','pax_fcny']
ymean = np.mean(data[int_col][data[int_col]!=0])#计算除0以外的平均值
ystd = np.std(data[int_col][data[int_col]!=0])#计算除0以外的标准差
threshold1 = ymean - n * ystd
threshold2 = ymean + n * ystd
for i in int_col:
    data[i][data[i] < threshold1[i]]=threshold1[i]#下界溢出部分转化
    data[i][data[i] > threshold2[i]]=threshold2[i]#上界界溢出部分转化

/Users/hanpeitao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hanpeitao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


# 偏爱机型

In [11]:
m3a_col = []#利用正则选择相关特征列
m6a_col = []
y1a_col = []
y2a_col = []
y3a_col = []
for col in data.columns:
    res_m3a = re.search(r'pref_aircraft_m3',col)
    res_m6a = re.search(r'pref_aircraft_m6',col)
    res_y1a = re.search(r'pref_aircraft_y1',col)
    res_y2a = re.search(r'pref_aircraft_y2',col)
    res_y3a = re.search(r'pref_aircraft_y3',col)
    if res_m3a:
        #print(res.group())
        m3a_col.append(col)
    elif res_m6a:
        m6a_col.append(col)
    elif res_y1a:
        y1a_col.append(col)
    elif res_y2a:
        y2a_col.append(col)
    elif res_y3a:
        y3a_col.append(col)

In [12]:
m3a_col

['pref_aircraft_m3_1',
 'pref_aircraft_m3_2',
 'pref_aircraft_m3_3',
 'pref_aircraft_m3_4',
 'pref_aircraft_m3_5']

In [13]:
a_col=[m3a_col[0],m6a_col[0],y1a_col[0],y2a_col[0],y3a_col[0]]
df_per_air=pd.DataFrame(data[a_col])
df_per_air

,pref_aircraft_m3_1,pref_aircraft_m6_1,pref_aircraft_y1_1,pref_aircraft_y2_1,pref_aircraft_y3_1
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,773
4,0,0,0,773,773
...,...,...,...,...,...
23427,0,0,0,0,0
23428,0,0,0,0,0
23429,0,0,0,0,0
23430,0,0,0,0,0


In [14]:
#只选用第一偏好
a_col=[m3a_col[0],m6a_col[0],y1a_col[0],y2a_col[0],y3a_col[0]]
df_per_air=pd.DataFrame(data[a_col])

In [15]:
import time
start=time.time()
f_per_air=[]
for index in data.index:
    res=df_per_air.loc[index].value_counts().index
    if res[0]=='0' and len(res)>1:
        air=res[1]
    else:
        air=res[0]
    #air=res[0]
    f_per_air.append(air)
end=time.time()
end-start

15.9775550365448

In [16]:
data['pref_aircraft']=f_per_air
data.drop(y3a_col+y2a_col+y1a_col+m3a_col+m6a_col,axis=1,inplace=True)

# 偏好

In [17]:
orig_col = []#利用正则选择相关特征列
line_col = []
city_col = []
month_col = []
dest_col = []
chnl_col=[]
for col in data.columns:
    res_orig = re.search(r'pref_orig',col)
    res_line = re.search(r'pref_line',col)
    res_city = re.search(r'pref_city',col)
    res_month = re.search(r'pref_month',col)
    res_dest = re.search(r'pref_dest',col)
    res_chnl = re.search(r'pit_add_chnl',col)
    if res_orig:
        #print(res.group())
        orig_col.append(col)
    elif res_line:
        line_col.append(col)
    elif res_city:
        city_col.append(col)
    elif res_month:
        month_col.append(col)
    elif res_dest:
        dest_col.append(col)
    elif res_chnl:
        chnl_col.append(col)

In [18]:
chnl_col

['pit_add_chnl_m3',
 'pit_add_chnl_m6',
 'pit_add_chnl_y1',
 'pit_add_chnl_y2',
 'pit_add_chnl_y3']

In [19]:
orig_col_1=[orig_col[0],orig_col[5],orig_col[10],orig_col[15],orig_col[20]]
orig_col_1

['pref_orig_m3_1',
 'pref_orig_m6_1',
 'pref_orig_y1_1',
 'pref_orig_y2_1',
 'pref_orig_y3_1']

In [20]:
orig_df=pd.DataFrame(data[orig_col_1])
orig_df2=pd.DataFrame(data[orig_col[-5:]])

In [21]:
start=time.time()
per_orig=[]
for index in data.index:
    res=orig_df.loc[index].value_counts().index
    if res[0]=='0' and len(res)>1:
        city=res[1]
    else:
        city=res[0]
    #air=res[0]
    per_orig.append(city)
end=time.time()
end-start

15.961233139038086

In [22]:
#start=time.time()
per_orig_city=[]
for index in data.index:
    res=orig_df2.loc[index].value_counts().index
    if res[0]=='0' and len(res)>1:
        city=res[1]
    else:
        city=res[0]
    #air=res[0]
    per_orig_city.append(city)
#end=time.time()
end-start

15.961233139038086

In [23]:
dest_df=pd.DataFrame(data[dest_col])
dest_df

,pref_dest_city_m3,pref_dest_city_m6,pref_dest_city_y1,pref_dest_city_y2,pref_dest_city_y3
0,0,0,0,0,0
1,0,0,New York,New York,New York
2,0,0,0,0,0
3,0,0,0,0,Los Angeles
4,0,0,0,0,0
...,...,...,...,...,...
23427,0,0,0,0,0
23428,0,0,0,0,0
23429,0,0,0,0,0
23430,0,0,0,0,0


In [25]:
#以per_dest_city为例（其他非偏爱类型的字符串指标处理均于此类似）
per_dest_city=[]
for index in data.index:
    res=dest_df.loc[index].value_counts().index
    if res[0]=='0' and len(res)>1:
        city=res[1]
    else:
        city=res[0]
    #air=res[0]
    per_dest_city.append(city)

In [26]:
chnl_df=pd.DataFrame(data[chnl_col])
chnl_df

,pit_add_chnl_m3,pit_add_chnl_m6,pit_add_chnl_y1,pit_add_chnl_y2,pit_add_chnl_y3
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
23427,0,0,0,0,0
23428,0,0,0,0,0
23429,0,0,0,0,0
23430,0,0,0,0,0


In [27]:
start=time.time()
pit_add_chnl=[]
for index in data.index:
    res=chnl_df.loc[index].value_counts().index
    if res[0]=='0' and len(res)>1:
        city=res[1]
    else:
        city=res[0]
    #air=res[0]
    pit_add_chnl.append(city)
end=time.time()
end-start

16.17685294151306

In [28]:
city_col_1=[city_col[0],city_col[5],city_col[10],city_col[15],city_col[20]]
city_col_1

['pref_city_m3_1',
 'pref_city_m6_1',
 'pref_city_y1_1',
 'pref_city_y2_1',
 'pref_city_y3_1']

In [29]:
city_df=pd.DataFrame(data[city_col_1])
city_df

,pref_city_m3_1,pref_city_m6_1,pref_city_y1_1,pref_city_y2_1,pref_city_y3_1
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,PVG
4,0,0,0,PVG,PVG
...,...,...,...,...,...
23427,0,0,0,0,0
23428,0,0,0,0,0
23429,0,0,0,0,0
23430,0,0,0,0,0


In [30]:
start=time.time()
per_city=[]
for index in data.index:
    res=city_df.loc[index].value_counts().index
    if res[0]=='0' and len(res)>1:
        city=res[1]
    else:
        city=res[0]
    #air=res[0]
    per_city.append(city)
end=time.time()
end-start

15.978559970855713

In [31]:
month_col_1=[month_col[0],month_col[5],month_col[10],month_col[15],month_col[20]]
month_col_1

['pref_month_m3_1',
 'pref_month_m6_1',
 'pref_month_y1_1',
 'pref_month_y2_1',
 'pref_month_y3_1']

In [32]:
month_df=pd.DataFrame(data[month_col_1])
month_df

,pref_month_m3_1,pref_month_m6_1,pref_month_y1_1,pref_month_y2_1,pref_month_y3_1
0,0,0,0,0,0
1,0,0,2,2,2
2,0,0,0,0,0
3,0,0,0,0,12
4,0,0,0,0,0
...,...,...,...,...,...
23427,0,0,0,0,0
23428,0,0,0,0,0
23429,0,0,0,0,0
23430,0,0,0,0,0


In [33]:
start=time.time()
per_month=[]
for index in data.index:
    res=month_df.loc[index].value_counts().index
    if res[0]=='0' and len(res)>1:
        city=res[1]
    else:
        city=res[0]
    #air=res[0]
    per_month.append(city)
end=time.time()
end-start

15.69398307800293

In [34]:
line_col_1=[line_col[0],line_col[5],line_col[10],line_col[15],line_col[20]]
line_col_1

['pref_line_m3_1',
 'pref_line_m6_1',
 'pref_line_y1_1',
 'pref_line_y2_1',
 'pref_line_y3_1']

In [35]:
line_df=pd.DataFrame(data[line_col_1])
line_df

,pref_line_m3_1,pref_line_m6_1,pref_line_y1_1,pref_line_y2_1,pref_line_y3_1
0,0,0,0,0,0
1,0,0,TPE-PVG,TPE-PVG,TPE-PVG
2,0,0,0,0,0
3,0,0,0,0,LAX-PVG
4,0,0,0,0,0
...,...,...,...,...,...
23427,0,0,0,0,0
23428,0,0,0,0,0
23429,0,0,0,0,0
23430,0,0,0,0,0


In [36]:
start=time.time()
per_line=[]
for index in data.index:
    res=line_df.loc[index].value_counts().index
    if res[0]=='0' and len(res)>1:
        city=res[1]
    else:
        city=res[0]
    #air=res[0]
    per_line.append(city)
end=time.time()
end-start

15.88824200630188

In [37]:
data.drop(orig_col+line_col+month_col+dest_col+city_col+chnl_col,axis=1,inplace=True)

In [38]:
data['per_city']=per_city
data['per_month']=per_month
data['per_line']=per_line
data['per_dest_city']=per_dest_city
data['per_orig_city']=per_orig_city
data['pit_add_chnl']=pit_add_chnl

# 指标的提取

In [40]:
data['seg_dep_time_loc']=pd.to_datetime(data['seg_dep_time'])
print(data['seg_dep_time'])
max_date=pd.Timestamp(data['seg_dep_time_loc'].max()).timestamp()
date_df=data['seg_dep_time_loc'].apply(lambda x:pd.Timestamp(x).timestamp())
data['seg_dep_time_interval']=data['seg_dep_time_loc'].apply(lambda x:x.hour)
data.drop('seg_dep_time',axis=1,inplace=True)
data['seg_dep_time_loc']

0        2018/12/16 20:00
1         2019/2/18 20:00
2          2018/3/9 13:20
3          2018/1/2 13:20
4          2018/7/6 11:30
               ...       
23427      2017/8/1 20:20
23428     2017/3/25 20:20
23429     2017/3/25 20:20
23430     2017/3/25 20:20
23431     2017/2/26 20:20
Name: seg_dep_time, Length: 23432, dtype: object


0       2018-12-16 20:00:00
1       2019-02-18 20:00:00
2       2018-03-09 13:20:00
3       2018-01-02 13:20:00
4       2018-07-06 11:30:00
                ...        
23427   2017-08-01 20:20:00
23428   2017-03-25 20:20:00
23429   2017-03-25 20:20:00
23430   2017-03-25 20:20:00
23431   2017-02-26 20:20:00
Name: seg_dep_time_loc, Length: 23432, dtype: datetime64[ns]

In [41]:
data['seg_dep_time_loc']=max_date-date_df
data['seg_dep_time_gap']=data['seg_dep_time_loc']/(24*60*60)
#data['seg_dep_time_gap']=data['seg_dep_time_gap'].astype(int)
data.drop('seg_dep_time_loc',axis=1,inplace=True)

In [43]:
df=data[['seg_dep_time_interval','seg_dep_time_gap']]
df['passport']=passport
df.to_csv('time_df.csv',index=False)

/Users/hanpeitao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
data.drop(['pax_name','pax_passport','seg_flight','birth_date','recent_flt_day','emd_lable','ffp_nbr'],axis=1,inplace=True)
#删除无用的特征列
object_col = [col for col in data if data[col].dtypes==object]

# 将字符串类型的变为频率

In [47]:
data[object_col[1]].value_counts(1)

LAX    0.337573
JFK    0.336890
SYD    0.325538
Name: seg_route_to, dtype: float64

In [48]:
for col in object_col:
    dict_t = data[col].value_counts(1).to_dict()#获得类型频率
    if '0' in [i for i in dict_t]:
        dict_t['0']=0
    np.save('dict/dict_{}.npy'.format(col), dict_t)#储存频率数据
    data[col].replace(dict_t,inplace=True)

# 合并时间相关的变量

In [50]:
y3_col = []#dist_cnt_y3\flt_nature_cnt_y3没有m3,m6,需要单独处理
y2_col = []##利用正则选取不同时间范围的特征
y1_col = []
m6_col = []
m3_col = []
for col in data.columns:
    res_y3 = re.search(r'y3$',col)
    res_y2 = re.search(r'y2$',col)
    res_y1 = re.search(r'y1$',col)
    res_m6 = re.search(r'm6$',col)
    res_m3 = re.search(r'm3$',col)
    if res_y3:
        #print(res.group())
        y3_col.append(col)
    elif res_y2:
        y2_col.append(col)
    elif res_y1:
        y1_col.append(col)
    elif res_m6:
        m6_col.append(col)
    elif res_m3:
        m3_col.append(col)

In [51]:
y3_col.pop(y3_col.index('dist_cnt_y3'))
y2_col.pop(y2_col.index('dist_cnt_y2'))
y1_col.pop(y1_col.index('dist_cnt_y1'))
y3_col.pop(y3_col.index('flt_nature_cnt_y3'))
y2_col.pop(y2_col.index('flt_nature_cnt_y2'))
y1_col.pop(y1_col.index('flt_nature_cnt_y1'))

'flt_nature_cnt_y1'

In [52]:
y1_df = pd.DataFrame(data[y1_col])#建立df便于之后的计算
y2_df = pd.DataFrame(data[y2_col].values,columns=y1_col)
y3_df = pd.DataFrame(data[y3_col].values,columns=y1_col)
m3_df = pd.DataFrame(data[m3_col].values,columns=y1_col)
m6_df = pd.DataFrame(data[m6_col].values,columns=y1_col)

In [53]:
#异常值处理
for i in range(m6_df.shape[0]):
    for j in range(m6_df.shape[1]):
        if m3_df.iloc[i,j]>m6_df.iloc[i,j]:
            m6_df.iloc[i,j]=m3_df.iloc[i,j]
for i in range(m6_df.shape[0]):
    for j in range(m6_df.shape[1]):
        if y1_df.iloc[i,j]<m6_df.iloc[i,j]:
            y1_df.iloc[i,j]=m6_df.iloc[i,j]
for i in range(m6_df.shape[0]):
    for j in range(m6_df.shape[1]):
        if y2_df.iloc[i,j]<y1_df.iloc[i,j]:
            y2_df.iloc[i,j]=y1_df.iloc[i,j]
for i in range(m6_df.shape[0]):
    for j in range(m6_df.shape[1]):
        if y3_df.iloc[i,j]<y2_df.iloc[i,j]:
            y3_df.iloc[i,j]=y2_df.iloc[i,j]

In [54]:
#重要性权重计算
res1=m3_df*0.6*0.75*0.75+(m6_df-m3_df)*0.6*0.75*0.25+(y1_df-m6_df)*0.6*0.25+(y2_df-y1_df)*0.3+(y3_df-y2_df)*0.1

In [55]:
df=pd.DataFrame(passport.values,columns=['passport'])
df['m3']=m3_df['dist_all_cnt_y1']
df['m6']=m6_df['dist_all_cnt_y1']
df['y1']=y1_df['dist_all_cnt_y1']
df['y2']=y2_df['dist_all_cnt_y1']
df['y3']=y3_df['dist_all_cnt_y1']
df['dist_all_cnt']=res1['dist_all_cnt_y1']
df

,passport,m3,m6,y1,y2,y3,dist_all_cnt
0,93d0aceeaef3f85264e5c5b5b1148a90,0.0,0.0,0,0.0,0.0,0.0
1,6930f6fa59d86c11fa28c610ab6f56a5,0.0,0.0,0,0.0,0.0,0.0
2,da73afa8f6ededd29d67a9331dccdbe9,0.0,0.0,0,0.0,0.0,0.0
3,dc31eead23771810dbb586ad236ade3b,0.0,0.0,0,0.0,21675.0,2167.5
4,7689603247539830a08d945cb1a57823,0.0,0.0,0,28705.0,28705.0,8611.5
...,...,...,...,...,...,...,...
23427,e61b68ce193bce838436620230ce94e1,0.0,0.0,0,0.0,0.0,0.0
23428,fdd824109fccadfbcba782aaae72484f,0.0,0.0,0,0.0,0.0,0.0
23429,ef53ebb9358d5c7b013f6aed92d78a42,0.0,0.0,0,0.0,0.0,0.0
23430,5fe336074ba73e83e12dccb31c33fe40,0.0,0.0,0,0.0,0.0,0.0


In [56]:
df.to_csv('d-a-c_df.csv',encoding='UTF-8',index=False)

In [57]:
dist_col = []#dist_cnt_y3\flt_nature_cnt_y3没有m3,m6,需要单独处理
flt_col = []
for col in data.columns:
    res_y3 = re.search(r'^dist_cnt',col)
    res_y2 = re.search(r'^flt_nature_cnt',col)
    if res_y3:
        #print(res.group())
        dist_col.append(col)
    elif res_y2:
        flt_col.append(col)

In [58]:
dist_df = pd.DataFrame(data[dist_col])#同上
flt_df = pd.DataFrame(data[flt_col])

In [59]:
dist_df['dist_cnt_y2']=dist_df['dist_cnt_y1']#填补y2缺失值
flt_df['flt_nature_cnt_y2']=flt_df['flt_nature_cnt_y1']

In [60]:
#重要性权重计算
res2=dist_df[dist_col[0]]*0.6+(dist_df[dist_col[1]]-dist_df[dist_col[0]])*0.3+(dist_df[dist_col[2]]-dist_df[dist_col[1]])*0.1
res3=flt_df[flt_col[0]]*0.6+(flt_df[flt_col[1]]-flt_df[flt_col[0]])*0.3+(flt_df[flt_col[2]]-flt_df[flt_col[1]])*0.1
data.drop(y3_col+y2_col+y1_col+m3_col+m6_col+dist_col+flt_col,axis=1,inplace=True)

In [61]:
data=pd.concat([data,res1],axis=1)

In [62]:
data['dist_cnt']=res2
data['flt_natrue_cnt']=res3
pd.concat([dist_df,data['dist_cnt'],flt_df,data['flt_natrue_cnt']],axis=1).to_csv('dist_flt.csv',index=False)

In [63]:
data

,seg_route_from,seg_route_to,seg_cabin,pax_fcny,pax_tax,emd_lable2,gender,age,residence_country,nation_name,...,pit_avg_amt_y1,pit_avg_interval_y1,pit_ech_avg_amt_y1,pit_out_avg_amt_y1,pit_income_cnt_y1,pit_pay_cnt_y1,pit_income_avg_amt_y1,pit_pay_avg_amt_y1,dist_cnt,flt_natrue_cnt
0,1.0,0.336890,0.126408,4161.0,584.43,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.336890,0.126408,1670.0,248.80,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.337573,0.869025,243.0,301.31,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.337573,0.869025,1362.0,454.16,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7571.5,0.7
4,1.0,0.336890,0.869025,675.0,284.11,1,0.10379,0.0,0.183083,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2870.5,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23427,1.0,0.325538,0.869025,807.0,486.50,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23428,1.0,0.325538,0.126408,2218.0,312.90,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23429,1.0,0.325538,0.126408,2218.0,312.90,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23430,1.0,0.325538,0.126408,2218.0,312.90,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
df=pd.concat([dist_df,flt_df],axis=1)
df[['dist_cnt','flt_natrue_cnt']]=data[['dist_cnt','flt_natrue_cnt']]
df['pax_passport']=passport

# d3 d7 d14 d30 d99

In [66]:
d3_col = []#筛选列
d7_col = []
d14_col = []
d30_col = []
d99_col = []
for col in data.columns:
    res_d3 = re.search(r'd3$',col)
    res_d7 = re.search(r'd7$',col)
    res_d14 = re.search(r'd14$',col)
    res_d30 = re.search(r'd30$',col)
    res_d99 = re.search(r'd99$',col)
    if res_d3:
        #print(res.group())
        d3_col.append(col)
    elif res_d7:
        d7_col.append(col)
    elif res_d14:
        d14_col.append(col)
    elif res_d30:
        d30_col.append(col)
    elif res_d99:
        d99_col.append(col)

In [67]:
d3_df = pd.DataFrame(data[d3_col])
d7_df = pd.DataFrame(data[d7_col].values,columns=d3_col)
d14_df = pd.DataFrame(data[d14_col].values,columns=d3_col)
d30_df = pd.DataFrame(data[d30_col].values,columns=d3_col)
d99_df = pd.DataFrame(data[d99_col].values,columns=d3_col)

In [68]:
d_df=d99_df*0.6*0.75*0.75+d30_df*0.6*0.75*0.25+d14_df*0.6*0.25+d7_df*0.3+d3_df*0.1

In [69]:
y3_col = []
y2_col = []
y1_col = []
m6_col = []
m3_col = []
for col in d_df.columns:
    res_y3 = re.search(r'y3',col)
    res_y2 = re.search(r'y2',col)
    res_y1 = re.search(r'y1',col)
    res_m6 = re.search(r'm6',col)
    res_m3 = re.search(r'm3',col)
    if res_y3:
        #print(res.group())
        y3_col.append(col)
    elif res_y2:
        y2_col.append(col)
    elif res_y1:
        y1_col.append(col)
    elif res_m6:
        m6_col.append(col)
    elif res_m3:
        m3_col.append(col)

In [70]:
y1_2_df = pd.DataFrame(data[y1_col])
y2_2_df = pd.DataFrame(data[y2_col].values,columns=y1_col)
y3_2_df = pd.DataFrame(data[y3_col].values,columns=y1_col)
m3_2_df = pd.DataFrame(data[m3_col].values,columns=y1_col)
m6_2_df = pd.DataFrame(data[m6_col].values,columns=y1_col)

In [71]:
res4=m3_2_df*0.6*0.75*0.75+(m6_2_df-m3_2_df)*0.6*0.75*0.25+(y1_2_df-m6_2_df)*0.6*0.25+(y2_2_df-y1_2_df)*0.3+(y3_2_df-y2_2_df)*0.1

In [72]:
data.drop(d3_col+d7_col+d14_col+d30_col+d99_col,axis=1,inplace=True)

In [73]:
data=pd.concat([data,res4],axis=1)

In [74]:
data

,seg_route_from,seg_route_to,seg_cabin,pax_fcny,pax_tax,emd_lable2,gender,age,residence_country,nation_name,...,pit_ech_avg_amt_y1,pit_out_avg_amt_y1,pit_income_cnt_y1,pit_pay_cnt_y1,pit_income_avg_amt_y1,pit_pay_avg_amt_y1,dist_cnt,flt_natrue_cnt,prebuy_d_cnt_y1_d3,prebuy_i_cnt_y1_d3
0,1.0,0.336890,0.126408,4161.0,584.43,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.336890,0.126408,1670.0,248.80,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.337573,0.869025,243.0,301.31,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.337573,0.869025,1362.0,454.16,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7571.5,0.7,0.0,0.0
4,1.0,0.336890,0.869025,675.0,284.11,1,0.10379,0.0,0.183083,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2870.5,0.4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23427,1.0,0.325538,0.869025,807.0,486.50,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23428,1.0,0.325538,0.126408,2218.0,312.90,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23429,1.0,0.325538,0.126408,2218.0,312.90,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23430,1.0,0.325538,0.126408,2218.0,312.90,0,0.00000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
df137=pd.concat([df137,data],axis=1)

# 删除指标

In [76]:
one_value_fea = [col for col in data.columns if data[col].nunique() <= 1]#所有数值都相同的特征
#data.drop([col for col in one_value_fea],axis=1,inplace=True)
one_value_fea

['seg_route_from',
 'often_city',
 'birth_interval_day',
 'tkt_avg_amt',
 'mdl_lost_idx',
 'mdl_loyal',
 'seat_safe_cnt_y1',
 'workday_cnt_y1',
 'weekend_cnt_y1',
 'holiday_cnt_y1',
 'close_holiday_cnt_y1',
 'rest_cnt_y1',
 'tkt_avg_disc_y1',
 'pit_add_mnl_amt_y1',
 'pit_add_ech_amt_y1',
 'pit_add_oth_amt_y1',
 'pit_des_tkt_amt_y1',
 'pit_des_bag_amt_y1',
 'pit_des_selt_amt_y1',
 'pit_des_oth_amt_y1',
 'pit_add_mnl_cnt_y1',
 'pit_add_ech_cnt_y1',
 'pit_add_oth_cnt_y1',
 'pit_des_tkt_cnt_y1',
 'pit_des_bag_cnt_y1',
 'pit_des_selt_cnt_y1',
 'pit_des_oth_cnt_y1']

In [78]:
data[one_value_fea].to_csv('one_value_fea.csv',index=False)
data.drop(one_value_fea,axis=1,inplace=True)
data[['seg_route_to', 'emd_lable2']] = data[['emd_lable2', 'seg_route_to']]
data.rename(columns={'seg_route_to':'emd_lable2','emd_lable2':'seg_route_to'},inplace=True)

# 数值标准化与归一化

In [79]:
#z-score标准化
data = (data-data.mean())/data.std()
#归一化
data = (data-data.min())/(data.max()-data.min())

# 数据平衡

In [82]:
data_emd1=data[data['emd_lable2']==1]
data1=pd.concat([data_emd1,data_emd1])
for i in range(13):
    data1=pd.concat([data1,data_emd1])

In [84]:
data=pd.concat([data[data['emd_lable2']==0],data1])
data

,emd_lable2,seg_cabin,pax_fcny,pax_tax,seg_route_to,gender,age,residence_country,nation_name,city_name,...,pit_ech_avg_amt_y1,pit_out_avg_amt_y1,pit_income_cnt_y1,pit_pay_cnt_y1,pit_income_avg_amt_y1,pit_pay_avg_amt_y1,dist_cnt,flt_natrue_cnt,prebuy_d_cnt_y1_d3,prebuy_i_cnt_y1_d3
0,0.0,0.145418,0.011063,0.002607,0.943262,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.145418,0.004440,0.001110,0.943262,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0.0,1.000000,0.000646,0.001344,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0.0,1.000000,0.003621,0.002026,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.032841,0.011041,0.0,0.0
5,0.0,1.000000,0.000665,0.001330,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23178,1.0,1.000000,0.004188,0.000667,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23193,1.0,1.000000,0.001662,0.000819,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23198,1.0,1.000000,0.001587,0.001851,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23199,1.0,1.000000,0.001587,0.001851,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.033963,0.014196,0.0,0.0


In [85]:
data.to_csv('data_oversampling.csv',index=False)